In [2]:
import gym
from gym import spaces
import pandas as pd
import sys
sys.path.append(r"../../../")
#sys.path.append(r"../utils/data_manipulation")
import numpy as np
from random import choices
from sklearn.ensemble import RandomForestClassifier
import torch
import torch.nn as nn
import torch.optim as optim

In [6]:
df = pd.read_csv(r"data\reproduction\DeppClinic_patient_data_all.csv")
df = df.fillna(0)
time1_df = df[df.measurement.isin(['Time 1'])]
time2_df = df[df.measurement.isin(['Time 2'])]
time3_df = df[df.measurement.isin(['Time 3'])]
reshaped = time2_df.merge(time3_df, on='id', suffixes=['_time_2', '_time_3'])
reshaped = reshaped.merge(time1_df, on='id')
reshaped['Y'] = reshaped['modcon_target_time_2'].astype(bool) | reshaped['modcon_target_time_3'].astype(bool)

KeyError: 'modcon_target_time_2'

In [3]:
class DiagnosticEnv(gym.Env):
    def __init__(self, df, patient_id, questions, max_questions=8):
        super(DiagnosticEnv, self).__init__()

        self.df = df
        self.patient_id = patient_id
        self.max_questions = max_questions

        # Define action and observation space
        self.question_list = questions
        self.action_space = spaces.Discrete(len(self.question_list))

        # Assuming answers are discrete choices, modify as needed
        self.observation_space = spaces.Dict({question: spaces.Discrete(5) for question in self.question_list}) 
        self.state = None
        self.reset()

    def reset(self):
        self.state = {question: None for question in self.question_list}
        self.counter = 0
        return self.state

    def step(self, action):
        question = self.question_list[action]
        answer = self.get_value(question)
        self.state[question] = answer

        self.counter += 1
        is_done = self.counter >= self.max_questions
        reward = self.calculate_reward() if is_done else 0

        return self.state, reward, is_done

    def get_value(self, question):
        value = self.df.query("id == @self.patient_id")[question].iloc[0]
        return value

    def calculate_reward(self):
        def create_Y(df, state, row):
            X_cols = []
            for k, v in state.items():
                if v is not None:
                    X_cols.append(k)
            Y = df['Y']
            X = df[X_cols]
            x_test = row[X_cols]
            y_test = row['Y']

            return X, Y.astype(int), x_test, y_test
        
        row = self.df.query("id == @self.patient_id")
        
        X, Y, x_test, y_test = create_Y(self.df, self.state, row)
        clf = RandomForestClassifier()

        clf.fit(X,Y)
        y_pred = clf.predict(x_test)
        score = (y_pred == y_test).astype(int)
        return score.values[0]

In [ ]:
class DQNAgent:
    def __init__(self, action_space, state_size, learning_rate=0.001, discount_factor=0.95):
        self.action_space = action_space
        self.state_size = state_size
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor

        # Create the neural network for Q-value estimation
        self.model = nn.Sequential(
            nn.Linear(state_size, 64),
            nn.ReLU(),
            nn.Linear(64, action_space.n)
        )

        # Use Adam optimizer for model training
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def choose_action(self, state, epsilon=0.1):
        return self.action_space.sample()


    def learn(self, state, action, reward, next_state, done):
        state_tensor = torch.tensor(state, dtype=torch.float32)
        next_state_tensor = torch.tensor(next_state, dtype=torch.float32)

        # Get current Q-value estimate for the state-action pair
        q_values = self.model(state_tensor)[0, action]

        # Calculate the Q-value target using Bellman equation
        if done:
            target = reward
        else:
            next_q_values = self.model(next_state_tensor).detach().max(0)[0]
            target = reward + self.discount_factor * next_q_values

        # Calculate the loss between the Q-value estimate and target
        loss = (target - q_values) ** 2

        # Perform backpropagation to update model parameters
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


In [5]:
choices(list(df.columns), k=16)


['sci_af_ca_7',
 'c_ssrs_8_intake',
 'sci_af_ca_9',
 'sci_af_ca_6',
 'c_ssrs_5_2weeks',
 'sci_af_ca_34',
 'sci_af_ca_7',
 'mfq_24',
 'mfq_21',
 'mfq_7',
 'mfq_7',
 'sci_af_ca_36',
 'sci_af_ca_21',
 'sci_af_ca_2',
 'chameleon_psychiatric_stu',
 'measurement']

In [6]:
questions = [
 'sci_af_ca_4',
 'sci_af_ca_35',
 'sci_af_ca_17',
 'sci_af_ca_28',
 'c_ssrs_2_life',
 'sci_af_ca_14']


In [7]:
# Set up environment (assuming env is your DiagnosticEnv instance)
env = DiagnosticEnv(reshaped, patient_id='S6442', questions=questions, max_questions=4)  # fill with your parameters

# Initialize agent
agent = QLearningAgent(env.action_space)


In [8]:
for patient_id in reshaped.id[:2]:

    total_episodes = 200
    # Set up environment (assuming env is your DiagnosticEnv instance)
    env = DiagnosticEnv(reshaped, patient_id=patient_id, questions=questions, max_questions=4)  # fill with your parameters

    # Training loop
    for episode in range(total_episodes):
        state = env.reset()
        done = False

        while not done:
            action = agent.choose_action(str(state))  # State converted to string for simplicity
            print(f"{action = }")
            next_state, reward, done = env.step(action)
            agent.learn(str(state), action, reward, str(next_state))
            state = next_state

action = 4
action = 3
action = 0
action = 2


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future

action = 2
action = 1
action = 0
action = 3


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

action = 1
action = 1
action = 3
action = 0


...

C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

action = 3
action = 0
action = 4
action = 3


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future

action = 3
action = 0
action = 3
action = 3


C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
C:\Users\nogur\anaconda3\envs\DeppClinic\Lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [10]:
[key for key in agent.q_table.keys() if " 'c_ssrs_2_life': 1.0" in key]

["{'sci_af_ca_4': None, 'sci_af_ca_35': None, 'sci_af_ca_17': None, 'sci_af_ca_28': None, 'c_ssrs_2_life': 1.0, 'sci_af_ca_14': None}",
 "{'sci_af_ca_4': None, 'sci_af_ca_35': None, 'sci_af_ca_17': None, 'sci_af_ca_28': 2.0, 'c_ssrs_2_life': 1.0, 'sci_af_ca_14': None}",
 "{'sci_af_ca_4': 4.0, 'sci_af_ca_35': None, 'sci_af_ca_17': None, 'sci_af_ca_28': 2.0, 'c_ssrs_2_life': 1.0, 'sci_af_ca_14': None}",
 "{'sci_af_ca_4': 4.0, 'sci_af_ca_35': None, 'sci_af_ca_17': 4.0, 'sci_af_ca_28': 2.0, 'c_ssrs_2_life': 1.0, 'sci_af_ca_14': None}",
 "{'sci_af_ca_4': None, 'sci_af_ca_35': None, 'sci_af_ca_17': 4.0, 'sci_af_ca_28': None, 'c_ssrs_2_life': 1.0, 'sci_af_ca_14': 4.0}",
 "{'sci_af_ca_4': None, 'sci_af_ca_35': None, 'sci_af_ca_17': 4.0, 'sci_af_ca_28': None, 'c_ssrs_2_life': 1.0, 'sci_af_ca_14': None}",
 "{'sci_af_ca_4': None, 'sci_af_ca_35': 4.0, 'sci_af_ca_17': None, 'sci_af_ca_28': 2.0, 'c_ssrs_2_life': 1.0, 'sci_af_ca_14': None}",
 "{'sci_af_ca_4': None, 'sci_af_ca_35': 4.0, 'sci_af_ca_17'

In [11]:
only_c_ssrs_2_life = [key for key in agent.q_table.keys() if " 'c_ssrs_2_life': 1.0" in key][1]

In [12]:
agent.q_table[only_c_ssrs_2_life]

{0: 0, 1: 0, 2: 0, 3: 0.38868052779002565, 4: 0.10873016191471287, 5: 0}

In [12]:
only_c_ssrs_2_life

"{'sci_af_ca_4': None, 'sci_af_ca_35': None, 'sci_af_ca_17': None, 'sci_af_ca_28': None, 'c_ssrs_2_life': 1.0, 'sci_af_ca_14': None}"

In [13]:
env.question_list[3]

'sci_af_ca_28'

In [1]:
self.action_space.sample()

NameError: name 'self' is not defined